In [1]:
import pandas as pd
from underthesea import word_tokenize 
import string
from gensim.utils import simple_preprocess
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from nltk.util import ngrams

In [2]:
df = pd.read_csv("data.csv",encoding='utf-8')

In [3]:
def read_file(file):
    with open(file, 'r', encoding='utf-8') as f:
        line = f.readlines()
        lines = list(m.replace("\n","") for m in line)
        # print(line)
    f.close()
    return lines

In [4]:
Content = read_file("ListContent.txt")
Content[0:2]

['mẫu đồng_hồ thông_minh galaxy gear samsung ảnh afp slashgear hôm tương_tác tv thông_minh galaxy gear đảm_nhiệm chức_năng kênh âm_lượng tắt tv galaxy gear khả_năng theo_dõi sức_khỏe tức_là thông_tin sức_khỏe của đeo thiết_bị phát tv nắm được sức_khỏe của được hiện galaxy gear có_thể làm_việc các thiết_bị di_động của samsung như galaxy note galaxy tab inch các thiết_bị như galaxy galaxy note sẽ có cập_nhật để tương_tác được với galaxy gear thành luân',
 'phó_tổng_thống mỹ công_du châu_bàn biển đông nhật ký thỏa_thuận quân_sự philippines thủ_tướng nhật shinzo abe dự_kiến hội_đàm hai lãnh_đạo tái cam_kết_thúc_đẩy hợp_tác song_phương tokyowashington kyodo abe sẽ trao_đổi với phó_tổng_thống mỹ về lập_trường của nhật bản tranh_chấp biển đông biển hoa đông chuyến công_du malaysia singapore và philippines của ông abe tăng_cường hợp_tác kinh_tế và an_ninh với các thành_viên của hiệp_hội các quốc_gia đông_nam asean diễn trong bối_cảnh yêu_sách chủ_quyền cứng_rắn của trung_quốc trên biển đông dấ

In [5]:
from collections import defaultdict
# nltk.download('stopwords')
import string
import nltk

word_counts = defaultdict(int)
for doc in Content:
    words = doc.translate(str.maketrans('','',string.punctuation)).lower().split()
    words = ['<s>'] + words
    for i in range(1,len(words)):
        prev_word = words[i-1]
        curr_word = words[i]
        word_counts[(prev_word,curr_word)] +=1

In [6]:
word_counts

defaultdict(int,
            {('<s>', 'mẫu'): 196,
             ('mẫu', 'đồnghồ'): 23,
             ('đồnghồ', 'thôngminh'): 57,
             ('thôngminh', 'galaxy'): 11,
             ('galaxy', 'gear'): 12,
             ('gear', 'samsung'): 3,
             ('samsung', 'ảnh'): 6,
             ('ảnh', 'afp'): 417,
             ('afp', 'slashgear'): 3,
             ('slashgear', 'hôm'): 2,
             ('hôm', 'tươngtác'): 1,
             ('tươngtác', 'tv'): 1,
             ('tv', 'thôngminh'): 4,
             ('gear', 'đảmnhiệm'): 1,
             ('đảmnhiệm', 'chứcnăng'): 10,
             ('chứcnăng', 'kênh'): 1,
             ('kênh', 'âmlượng'): 1,
             ('âmlượng', 'tắt'): 1,
             ('tắt', 'tv'): 3,
             ('tv', 'galaxy'): 1,
             ('gear', 'khảnăng'): 1,
             ('khảnăng', 'theodõi'): 15,
             ('theodõi', 'sứckhỏe'): 58,
             ('sứckhỏe', 'tứclà'): 1,
             ('tứclà', 'thôngtin'): 1,
             ('thôngtin', 'sứckhỏe'): 7,
     

In [7]:
total = sum(word_counts.values())
total

17201914

In [8]:
# Compute the probabilities of each n-gram using add-k smoothing
k = 1
n = 2
# Compute the probabilities of each n-gram using Laplace smoothing
probs_laplace = defaultdict(lambda: defaultdict(float))
probs = defaultdict(lambda: defaultdict(float))

update_keys = []
a = word_counts.copy()
for gram in a.keys():
    value = word_counts[gram]
    count = word_counts.get(gram, 0)
    context = gram[:-1]
    update_keys.append(gram)
    probs_laplace[gram] = (count + k) / (word_counts[context] + k*total)
for key in update_keys:
    probs[key] = probs_laplace[key]


In [9]:
def sinhchu(input,probs):
    next_word_probs = defaultdict(float)
    context = tuple(input_tokens[-1:])
    for gram, prob in probs.items():
        if gram[:-1] == context:
            next_word = gram[-1]
            next_word_probs[next_word] += prob  
    return max(next_word_probs, key=next_word_probs.get)

In [10]:

input_phrase = "hôm nay trời học"
input_tokens = word_tokenize(input_phrase)

for i in range(10):
    a = sinhchu(input_tokens,probs)
    input_tokens.append(a)
print(input_tokens)

['hôm nay', 'trời', 'học', 'lớp', 'trường', 'đạihọc', 'caođẳng', 'sưphạm', 'kỹthuật', 'và', 'các', 'trường', 'đạihọc']


In [11]:
output_phrase = " ".join([a for a in input_tokens])
output_phrase

'hôm nay trời học lớp trường đạihọc caođẳng sưphạm kỹthuật và các trường đạihọc'

In [1]:
pip install transformers -U 

  Using cached transformers-4.27.1-py3-none-any.whl (6.7 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Computer-_-\anaconda3\python.exe -m pip install --upgrade pip' command.


In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(sentence)])

with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
text = 'Hôm nay trời đẹp quá.'
encoded_text = tokenizer.encode(text)
input_ids = torch.tensor(encoded_text).unsqueeze(0)
outputs = phobert(input_ids)